# Assignment 4: Evaluating Search Engines

For this assignment, we leave aside the code we developed so far, and look into the more general issue of how to evaluate and compare different search engines. The ultimate test for any Information Retrieval system is how well it is able to satisfy the information needs of users.

# Cohen's Kappa

Our evaluation will involve the calculation of [Cohen's Kappa](https://en.wikipedia.org/wiki/Cohen's_kappa) to quantify the degree to which two human assessors agree or disagree on whether results are considered relevant or not. To calculate Cohen's Kappa, we are going to use the [scikit-learn library](http://scikit-learn.org/stable/):

In [1]:
! pip install --user scikit-learn

In [2]:
from sklearn.metrics import cohen_kappa_score

This library expects relevance assessments as lists of elements where `1` stands for _relevant_ and `0` stands for _not relevant_, for example like this:

In [3]:
a1=[1,0,1,0,1,0,1,0]

This list means that the first document was assessed to be relevant, the second to be not relevant, the third to be relevant etc.

We need two assessments in order to calculate Cohen's Kappa, so let's make another exemplary list that only differs on the last element:

In [4]:
a2=[1,0,1,0,1,0,1,1]

We can now invoke the library as follows to calculate the agreement between the two:

In [5]:
cohen_kappa_score(a1, a2)

0.75

This value represents high agreement. We can reach maximal agreement if the two assessments are identical:

In [6]:
cohen_kappa_score(a1, a1)

1.0

Now, let's see what happens for a third assessment that differs on three positions with the first one (the three last positions):

In [7]:
a3=[1,0,1,0,1,1,0,1]

cohen_kappa_score(a1, a3)

0.25

We get a smaller but still positive value, because these two assessments still mostly agree. If we make a further example that differs on 6 of the 8 positions, we get the following result:

In [8]:
a4=[1,0,0,1,0,1,0,1]

cohen_kappa_score(a1, a4)

-0.5

The score is now negative, because the two differ on more positions than they agree. The agreement is in fact less than what you would expect to occur just by chance. We get the maximal disagreement if we define a fifth example that disagrees on all positions:

In [9]:
a5=[0,1,0,1,0,1,0,1]

cohen_kappa_score(a1, a5)

-1.0

Now that we understand how this function works, we will apply it below for our specific evaluation.

# Results and Assessments

Next, we will define some auxilary code to deal with lists of URLs from search engines and associated relevance assessments. We will encode result lists like this:

In [10]:
urls = [
    'https://en.wikipedia.org/wiki/Information_retrieval/',  # 1st result
    'http://www.dictionary.com/browse/information',          # 2nd result
    'https://nlp.stanford.edu/IR-book/'                      # ...
]

And we represent corresponding assessments, as above, as lists of the same size containing relevance values:

In [11]:
my_assessment = [1, 0, 1]
another_assessment = [0, 0, 1]

In order to nicely display URL lists, with or without related assessments, we define a function called `display_results`:

In [12]:
from IPython.display import display, HTML

def display_results(urls, assessment1=None, assessment2=None):
    lines = []
    lines.append('<table>')
    header = '<tr><th>#</th><th>Result URL</th>'
    if (assessment1):
        header += '<th>Assessment 1</th>'
    if (assessment2):
        header += '<th>Assessment 2</th>'
    header += '</tr>'
    lines.append(header)
    i = 0
    for url in urls:
        show_url = url
        if (len(url) > 80):
            show_url = url[:75] + '...'
        line = '<tr><td>{}</td><td><a href="{:s}">{:s}</a></td>'.format(i+1, url, show_url)
        if (assessment1):
            if (assessment1[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        if (assessment2):
            if (assessment2[i] == 0):
                line += '<td><em>Not relevant</em></td>'
            else:
                line += '<td><strong>Relevant</strong></td>'
        line += '</tr>'
        lines.append(line)
        i = i+1
    lines.append('</table>')
    display( HTML(''.join(lines)) )

We can use this function to display a list of URLs, optionally together with one or two assessment lists:

In [13]:
print("Just a list of URLs:")
display_results(urls)

print("With one assessment:")
display_results(urls, my_assessment)

print("With two assessments:")
display_results(urls, my_assessment, another_assessment)

Just a list of URLs:


#,Result URL
1,https://en.wikipedia.org/wiki/Information_retrieval/
2,http://www.dictionary.com/browse/information
3,https://nlp.stanford.edu/IR-book/


With one assessment:


#,Result URL,Assessment 1
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant
2,http://www.dictionary.com/browse/information,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant


With two assessments:


#,Result URL,Assessment 1,Assessment 2
1,https://en.wikipedia.org/wiki/Information_retrieval/,Relevant,Not relevant
2,http://www.dictionary.com/browse/information,Not relevant,Not relevant
3,https://nlp.stanford.edu/IR-book/,Relevant,Relevant


Now we are ready to perform an actual evaluation, which will involve a substantial amount of manual work.

---

# Tasks

**Your name:** Andrew Harrison

### Task 1

Think up and formulate an information need in the areas of Computer Science or the Life Sciences (medicine, biology, etc.) for which you think the answer can be found in scientific publications. On page 152 in the book an example of such an information need is shown: "Information on whether drinking red wine is more effective at reducing your risk of heart attacks than white wine."

**Answer:** _Information on whether Artificial Intelligence and Machine Learning academic communities are conducting research on privacy and data protection issues_

Next, write down specifically what documents have to look like to satisfy your information need. For example if your information need is about finding an overview of different cancer types, you could state that a document would need to list at least ten types of cancer to satisfy your information need (among other criteria). Write this down as a protocol with rules and examples. For example, such a protocol could state that at least three out of five given criteria have to be fulfilled for a document to be considered relevant for the information need, and then specify the criteria. Or your protocol could have the form of a sequence of rules, where each rule lets you either label the document as relevant or not relevant, or proceed with the next rule. Such rules and criteria can, for example, be about the general topic of the paper, the concepts mentioned in it, the covered relations between concepts, the type of publication (research paper, overview paper, etc.), the number of references, the types of contained diagrams, and so on, depending on your specified information need.

**Answer:** _The protocol contains a sequence of two rules, the first of which is that the document needs to contain either Artificial Intelligence or Machine Learning. The second rule, is that the document also needs to contain privacy or data protection related words (further details below). There are many documents that either of these rules would catch, but the intersection of both should hopefully limit the scope of returned documents those serving the information need.
Artificial Intelligence and Machine Learning research uses a multitude of specific and specalised terms, focusing for example on multi-agent learning, or robotic navigation, or NLP, or machine vision. However, as Artificial Intelligence is so in vogue, and the subfield of Machine Learning is currently dominant in AI, either of these terms are likely to be mentioned in relevant papers. AI is also the higher level grouping term.
For privacy and data, these are very exacting terms. Privacy is a human right, and very specifically used. Data protection is more multifarious, hence data privacy, data processing, and GDPR (newer EU legislation) terms are also used. Thus the second rule should also catch most papers that are researching these areas._

### Task 2

Formulate a keyword query that represents the information need. For the example on page 152 in the book (see above), the example query "wine AND red AND white AND heart AND attack AND effective" is given. (You don't need to use connectors like "AND", but if you do, make first sure your chosen search engines below actually support them.)

**Answer:** _("artificial intelligence" OR "AI" OR "machine learning" OR "ML") AND (privacy OR "data privacy" OR "data protection" OR "data processing" OR GDPR)_

Then submit your query to **two** of the following academic search engines:

- [Google Scholar](https://scholar.google.com) (all science disciplines)
- [Semantic Scholar](https://www.semanticscholar.org) (all science disciplines)
- [PubMed Search](https://www.ncbi.nlm.nih.gov/pubmed) (Life Sciences / biomedicine)

The right choice of two from the three search engine depends on the topic of your information need. If your information need is in the Life Sciences and biomedicine, it's probably best to include PubMed Search, but otherwise you should pick Google Scholar and Semantic Scholar.

Extract a list of the top 10 URLs of the lists of each of the search engines
given the query. Try to access the resulting publications. For the publications
where that is not possible (because of dead links or because the publication is
pay-walled even within the VU network), exclude them from the list and add more publications to the end of
your list (that is, append results number 11, then 12, etc. to ensure you have
two lists of 10 publications each). In order to deal with paywalls, you should try accessing the articles from the VU network, use
[UBVU Off-Campus
Access](http://www.ub.vu.nl.vu-nl.idm.oclc.org/nl/faciliteiten/toegang-buiten-de-campus/index.aspx), or try to find the respective documents from alternative sources (Google Scholar, for example, is very good at finding free PDFs of articles). If you get fewer than 10 results for one of the search engines, modify the keyword query above to make it more inclusive, and then redo the steps of this task.

Store your two lists of URLs in the form of Python lists as introduced above. Then, use the `display_results` function to nicely display them.

In [14]:
# Create two of the lists below, depending on your chosen engines:

urls_google = [
    'https://ergodicity.net/2013/07/',  # 1st result
    'https://link-springer-com.vu-nl.idm.oclc.org/article/10.1186/s13634-016-0355-x',          # 2nd result
    'https://ieeexplore-ieee-org.vu-nl.idm.oclc.org/abstract/document/7958569',                     # 3rd result
    'https://arxiv.org/abs/1611.03814', # 4th result
    'https://ieeexplore-ieee-org.vu-nl.idm.oclc.org/abstract/document/6582713', # 5th result
    'https://books.google.com/books?hl=en&lr=&id=azyjBQAAQBAJ&oi=fnd&pg=PP1&dq=(%22artificial+intelligence%22+OR+%22AI%22+OR+%22machine+learning%22+OR+%22ML%22)+AND+(privacy+OR+%22data+privacy%22+OR+%22data+protection%22+OR+%22data+processing%22+OR+GDPR)&ots=nikG9cP8Hc&sig=rXKSapUp9psWT5Nj-HDHPeqfl0o', # 6th result
    'http://iot.stanford.edu/pubs/bost-learning-ndss15.pdf', # 7th result
    'https://dl.acm.org/citation.cfm?id=573193', # 8th result
    'https://www.sciencedirect.com/science/article/pii/0022519367900975', # 9th result
    'https://ieeexplore.ieee.org/abstract/document/5403147/', #10th result
]

urls_semantic = [
    'https://www.semanticscholar.org/paper/Privacy-preserving-distributed-mining-of-rules-on-Kantarcioglu-Clifton/7435b796a7b3136b772e0d8d826fa927c04c5e2e',  # 1st result
    'https://www.semanticscholar.org/paper/Transforming-data-to-satisfy-privacy-constraints-Iyengar/2676d77b4e4cc58250ed20b4f85576a9fb33ae5a',          # 2nd result
    'https://www.semanticscholar.org/paper/Data-privacy-through-optimal-k-anonymization-Bayardo-Agrawal/d429d387f94e16fe2fcf63224758df9b0ff81430',                      # 3rd result
    'https://www.semanticscholar.org/paper/Atomic-Data-and-Nuclear-Data-Tables-Sawey-Berrington/dcee8aeff596fb54fb649ed6633c9316f56db9b3', # 4th result
    'https://www.semanticscholar.org/paper/Our-Data%2C-Ourselves%3A-Privacy-Via-Distributed-Noise-Dwork-Kenthapadi/1808b64aec21863489f0fe66f250890a3ac2b843', # 5th result
    'https://www.semanticscholar.org/paper/Privacy-Preserving-Data-Mining-Lindell-Pinkas/9a1d75a5c79e603ab7ceecc4bb07ee736e402c18', # 6th result
    'https://www.semanticscholar.org/paper/On-the-Design-and-Quantification-of-Privacy-Data-Agrawal-Aggarwal/20780ad665e496aa128f82713bb78d13fd87cd0a', # 7th result
    'https://www.semanticscholar.org/paper/ON-DATA-BANKS-AND-PRIVACY-HOMOMORPHISMS-Rivest-Dertouzos/c365f01d330b2211e74069120e88cff37eacbcf5', # 8th result
    'https://www.semanticscholar.org/paper/Limiting-privacy-breaches-in-privacy-preserving-Evfimievski-Gehrke/03b01daccd140e7d65358f31f8c4472d18573a5a', # 9th result
    'https://www.semanticscholar.org/paper/Privacy-Preserving-Multi-keyword-Ranked-Search-over-Vishvapathi-Reddy/fe274555cc1dd0609b72f666d0aa063ab68106e6', #10th result
]
#urls_pubmed = ...

# Call display_results here
display_results(urls_google)
display_results(urls_semantic)

#,Result URL
1,https://ergodicity.net/2013/07/
2,https://link-springer-com.vu-nl.idm.oclc.org/article/10.1186/s13634-016-0355-x
3,https://ieeexplore-ieee-org.vu-nl.idm.oclc.org/abstract/document/7958569
4,https://arxiv.org/abs/1611.03814
5,https://ieeexplore-ieee-org.vu-nl.idm.oclc.org/abstract/document/6582713
6,https://books.google.com/books?hl=en&lr=&id=azyjBQAAQBAJ&oi=fnd&pg=PP1&dq=(...
7,http://iot.stanford.edu/pubs/bost-learning-ndss15.pdf
8,https://dl.acm.org/citation.cfm?id=573193
9,https://www.sciencedirect.com/science/article/pii/0022519367900975
10,https://ieeexplore.ieee.org/abstract/document/5403147/


#,Result URL
1,https://www.semanticscholar.org/paper/Privacy-preserving-distributed-mining...
2,https://www.semanticscholar.org/paper/Transforming-data-to-satisfy-privacy-...
3,https://www.semanticscholar.org/paper/Data-privacy-through-optimal-k-anonym...
4,https://www.semanticscholar.org/paper/Atomic-Data-and-Nuclear-Data-Tables-S...
5,https://www.semanticscholar.org/paper/Our-Data%2C-Ourselves%3A-Privacy-Via-...
6,https://www.semanticscholar.org/paper/Privacy-Preserving-Data-Mining-Lindel...
7,https://www.semanticscholar.org/paper/On-the-Design-and-Quantification-of-P...
8,https://www.semanticscholar.org/paper/ON-DATA-BANKS-AND-PRIVACY-HOMOMORPHIS...
9,https://www.semanticscholar.org/paper/Limiting-privacy-breaches-in-privacy-...
10,https://www.semanticscholar.org/paper/Privacy-Preserving-Multi-keyword-Rank...


### Task 3

Then, find a fellow student who will **independently**
assess the results as "relevant" or "not relevant" using the protocol that you
have defined above, and also help (at least) one other student for his/her
assessment. Write down their names here:

**Name of the student who assesses my results:** _Mathias Parisot_

**Name of the student who I help to assess his/her results:** _Mathias Parisot_

Show to the other assessor everything you have written down above for Tasks 1 and 2 (and you might also want to give him/her the PDFs you got for these papers to simplify the process).

You as assessors need to stick to the protocol you made in Task 1 and should not discuss with each other, especially when you doubt whether a result is relevant or not. Write down your assessments as lists of relevance values, as introduced above, and make sure they correctly map to the URLs by displaying them together with the `display_results` function.

To avoid problems with extreme results, mark in each list at least one paper as 'relevant' and at least one paper as 'not relevant'. That is, if all papers seem relevant, mark the one that seems least relevant 'not relevant', and conversely, if none of the papers seem relevant, mark the one that seems a bit more relevant than the others as 'relevant'.

In [15]:
# 0 = not relevant; 1 = relevant

# You only need to create 4 of the following 6 lists, again depending on which search engines you chose.

# Assessment 1 is from you:

assessment1_google = [0,1,1,1,1,0,1,0,0,0]
assessment1_semantic = [0,1,0,0,1,1,0,0,1,0]
#assessment1_pubmed = ...

# Assessment 2 is from your fellow student (don't show him/her your own assessment!):

assessment2_google = [0,0,1,1,1,0,1,0,0,0]
assessment2_semantic = [0,1,1,0,1,1,1,0,1,1]
#assessment2_pubmed = ...

# Call display_results here
display_results(urls_google,assessment1_google, assessment2_google)
display_results(urls_semantic, assessment1_semantic, assessment2_semantic)

#,Result URL,Assessment 1,Assessment 2
1,https://ergodicity.net/2013/07/,Not relevant,Not relevant
2,https://link-springer-com.vu-nl.idm.oclc.org/article/10.1186/s13634-016-0355-x,Relevant,Not relevant
3,https://ieeexplore-ieee-org.vu-nl.idm.oclc.org/abstract/document/7958569,Relevant,Relevant
4,https://arxiv.org/abs/1611.03814,Relevant,Relevant
5,https://ieeexplore-ieee-org.vu-nl.idm.oclc.org/abstract/document/6582713,Relevant,Relevant
6,https://books.google.com/books?hl=en&lr=&id=azyjBQAAQBAJ&oi=fnd&pg=PP1&dq=(...,Not relevant,Not relevant
7,http://iot.stanford.edu/pubs/bost-learning-ndss15.pdf,Relevant,Relevant
8,https://dl.acm.org/citation.cfm?id=573193,Not relevant,Not relevant
9,https://www.sciencedirect.com/science/article/pii/0022519367900975,Not relevant,Not relevant
10,https://ieeexplore.ieee.org/abstract/document/5403147/,Not relevant,Not relevant


#,Result URL,Assessment 1,Assessment 2
1,https://www.semanticscholar.org/paper/Privacy-preserving-distributed-mining...,Not relevant,Not relevant
2,https://www.semanticscholar.org/paper/Transforming-data-to-satisfy-privacy-...,Relevant,Relevant
3,https://www.semanticscholar.org/paper/Data-privacy-through-optimal-k-anonym...,Not relevant,Relevant
4,https://www.semanticscholar.org/paper/Atomic-Data-and-Nuclear-Data-Tables-S...,Not relevant,Not relevant
5,https://www.semanticscholar.org/paper/Our-Data%2C-Ourselves%3A-Privacy-Via-...,Relevant,Relevant
6,https://www.semanticscholar.org/paper/Privacy-Preserving-Data-Mining-Lindel...,Relevant,Relevant
7,https://www.semanticscholar.org/paper/On-the-Design-and-Quantification-of-P...,Not relevant,Relevant
8,https://www.semanticscholar.org/paper/ON-DATA-BANKS-AND-PRIVACY-HOMOMORPHIS...,Not relevant,Not relevant
9,https://www.semanticscholar.org/paper/Limiting-privacy-breaches-in-privacy-...,Relevant,Relevant
10,https://www.semanticscholar.org/paper/Privacy-Preserving-Multi-keyword-Rank...,Not relevant,Relevant


### Task 4

Compute Cohen's kappa to quantify how much the two assessors agreed. Use the function `cohen_kappa_score` demonstrated above to calculate two times the inter-annotator agreement (once for each of the two search engines), and print out the results.

In [16]:
# Add your code here:

kappa_google = cohen_kappa_score(assessment1_google, assessment2_google)
kappa_semantic = cohen_kappa_score(assessment1_semantic, assessment2_semantic)
#kappa_pubmed = ...

print("Kappa for Google Scholar:", kappa_google)
print("Kappa for Semantic Scholar:", kappa_semantic)
#print("Kappa for PubMed:", kappa_pubmed)

Kappa for Google Scholar: 0.8
Kappa for Semantic Scholar: 0.44444444444444453


Explain whether the agreement can be considered high or not, based on the interpretation table on [this Wikipedia page](https://en.wikipedia.org/wiki/Fleiss'_kappa#Interpretation) (this Wikipedia page is about a different type of kappa but the interpretation table can also be used for Cohen's kappa).

**Answer:** _Agreement for Google Scholar is considered "Substantial agreement" according to the interpretation table. Whereas agreement on Semantic Scholar is considered "Moderate agreement". Note the kappa will also be higher when the number of catergories are low (and here there are only 2), and that these intervals are just personal opinion and not based on evidence. So I would just be comfortable asserting that the Google Scholar agreement is high, but no so much that the Semantic Scholar agreement is. Note further we're not doing any statistical checks for significance on these values either._

### Task 5

Define a function called `precision_at_n` that calculates Precision@n as described in the lecture slides, which takes as input an assessment list and a value for _n_ and returns the respective Precision@n value. Run this function to calculate Precision@10 (that is, n=10) on all four assessments (two assessors and two search engines).

In [17]:
# Add your code here:

def precision_at_n(assessment_list, n):
    relevant = 0
    for assessment in assessment_list:
        if assessment == 1:
            relevant+=1
    return (relevant/n)

# Print out Precision@10 for all assessments here.
print('Precision@10 for asssesment1_google:',precision_at_n(assessment1_google,10))
print('Precision@10 for asssesment2_google:',precision_at_n(assessment2_google,10))
print('Precision@10 for asssesment1_semantic:',precision_at_n(assessment1_semantic,10))
print('Precision@10 for asssesment1_semantic:',precision_at_n(assessment2_semantic,10))

Precision@10 for asssesment1_google: 0.5
Precision@10 for asssesment2_google: 0.4
Precision@10 for asssesment1_semantic: 0.4
Precision@10 for asssesment1_semantic: 0.7


Explain what these specific Precision@10 results tell us (or don't tell us) about the quality of the two search engines for your particular information need. You can also refer to the results of Task 4 if necessary.

**Answer:** _We have no knowledge of total relevant documents, so cannot say anything about recall. We can however discuss the precision, as we have true positives and false positives. Thus why it's precision at 10. However, given there are multiple assessors, how to combine these values? For the particular information need, a simple average would give 0.45 for Google Scholar and 0.55 for Semantic Scholar, perhaps implying that Semantic Scholar is more precise. Or, you could take the lowest precision value from across the assessors for each search engine, giving both 0.4 precision and thus saying they're the same. Alternatively, you may  take the average and weight it by the level of agreement, hence 0.45*0.8=0.36 and 0.55*0.44=0.242 and hence say Google Scholar is more precise. Finally, Precision@10 doesn't say anything about the ordering of the results, as for Assesor 1 the first 5 Google Scholar gets 4 relevant out of 5, and Semantic only 2 out of 5 (albeit, you could reduce to precision@5). But staying with precision@10, this is information that may be relevant, but is not captured in the difference. Maybe the first 5 search results aren't relevant for one search engine, but 6-10 are. For another search engine, the first 5 results are all relevant, and none of 6-10 are. Both of these would have Precision@10 of 0.5 but there is a case to argue for the latter search engine being better. In summary, it tells us some things, that the search engines aren't returning total rubbish. But I would be hard pressed to choose a winner._

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.